In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import torch
import matplotlib.pyplot as plt
from src import models, data
from tqdm.auto import tqdm
import json
import os
import numpy as np
import copy


In [3]:
mt = models.load_model("gptj", fp16=True, device="cuda")

In [12]:
#####################################
relation_name = "country currency"
#####################################

In [14]:
dataset = data.load_dataset()
relation = dataset.filter(
    relation_names = [relation_name]
)[0]

In [15]:
from src.utils.sweep_utils import read_sweep_results, relation_from_dict

In [25]:
sweep_dict = read_sweep_results("../results/sweep-24-trials/gptj/country_currency")

--> ../results/sweep-24-trials/gptj/country_currency
    --> ../results/sweep-24-trials/gptj/country_currency/2_country_currency_seed_88810
        --> ../results/sweep-24-trials/gptj/country_currency/2_country_currency_seed_88810/args-20230606-213844.json
        --> ../results/sweep-24-trials/gptj/country_currency/2_country_currency_seed_88810/results_all.json
        --> ../results/sweep-24-trials/gptj/country_currency/2_country_currency_seed_88810/country_currency.json
    --> ../results/sweep-24-trials/gptj/country_currency/3_country_currency_seed_709106
        --> ../results/sweep-24-trials/gptj/country_currency/3_country_currency_seed_709106/args-20230607-005634.json
        --> ../results/sweep-24-trials/gptj/country_currency/3_country_currency_seed_709106/results_all.json
        --> ../results/sweep-24-trials/gptj/country_currency/3_country_currency_seed_709106/country_currency.json
    --> ../results/sweep-24-trials/gptj/country_currency/1_country_currency_seed_71745
      

In [27]:
relation_result = relation_from_dict(sweep_dict[relation_name])

In [28]:
trial_options = list(range(len(relation_result.trials)))
print(f"{trial_options=}")

layer_options = [layer.layer for layer in relation_result.trials[0].layers]
print(f"{layer_options=}")

rank_options = [rank.rank for rank in relation_result.trials[0].layers[0].result.ranks]
print(f"{rank_options=}")

trial_options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
layer_options=['emb', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
rank_options=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240]


In [29]:
relation_result.best_by_efficacy().__dict__

{'layer': 3,
 'beta': AggregateMetric(mean=0.3437500049670537, stdev=0.08454843411382523, stderr=0.017258376844182854, values=[0.5, 0.25, 0.25, 0.30000001192092896, 0.4000000059604645, 0.3499999940395355, 0.30000001192092896, 0.30000001192092896, 0.30000001192092896, 0.45000001788139343, 0.20000000298023224, 0.3499999940395355, 0.20000000298023224, 0.25, 0.3499999940395355, 0.4000000059604645, 0.5, 0.3499999940395355, 0.45000001788139343, 0.4000000059604645, 0.30000001192092896, 0.30000001192092896, 0.3499999940395355, 0.45000001788139343]),
 'recall': AggregateMetric(mean=0.6476973684210526, stdev=0.07518332304912685, stderr=0.015346731553099194, values=[0.8, 0.6111111111111112, 0.47368421052631576, 0.6842105263157895, 0.6, 0.631578947368421, 0.5789473684210527, 0.5263157894736842, 0.631578947368421, 0.7777777777777778, 0.6111111111111112, 0.65, 0.6111111111111112, 0.6111111111111112, 0.631578947368421, 0.6666666666666666, 0.7368421052631579, 0.6666666666666666, 0.7368421052631579, 0.

In [30]:
#########################################################################################################
TRIAL_NO = 10
RANK = 70
LAYER = 3
#########################################################################################################

In [31]:
layer_result = [layer for layer in relation_result.trials[TRIAL_NO].layers if layer.layer == LAYER][0]
rank_result = [rank for rank in layer_result.result.ranks if rank.rank == RANK][0]
rank_result

SweepRankResults(rank=70, efficacy=[0.8888888888888888, 0.9444444444444444, 0.9444444444444444], efficacy_successes=[EfficacyTestPair(source=RelationSample(subject='Australia', object='Dollar'), target=RelationSample(subject='Japan', object='Yen')), EfficacyTestPair(source=RelationSample(subject='Hungary', object='Forint'), target=RelationSample(subject='Switzerland', object='Franc')), EfficacyTestPair(source=RelationSample(subject='Japan', object='Yen'), target=RelationSample(subject='Hungary', object='Forint')), EfficacyTestPair(source=RelationSample(subject='Malaysia', object='Ringgit'), target=RelationSample(subject='United Arab Emirates', object='Dirham')), EfficacyTestPair(source=RelationSample(subject='Mexico', object='Peso'), target=RelationSample(subject='Singapore', object='Dollar')), EfficacyTestPair(source=RelationSample(subject='New Zealand', object='Dollar'), target=RelationSample(subject='United Arab Emirates', object='Dirham')), EfficacyTestPair(source=RelationSample(su

In [32]:
efficacy_successes = {s.target.subject : s for s in rank_result.efficacy_successes}

for beta_result in layer_result.result.betas:
    faithfulness_successes = beta_result.faithfulness_successes
    for sample in faithfulness_successes:
        if(sample.subject in efficacy_successes):
            print(f"Edit: {efficacy_successes[sample.subject].source} <to> {efficacy_successes[sample.subject].target} -- found in beta: {beta_result.beta}")
            efficacy_successes.pop(sample.subject)
        

Edit: United States -> Dollar <to> Japan -> Yen -- found in beta: 0.0
Edit: Saudi Arabia -> Riyal <to> Malaysia -> Ringgit -- found in beta: 0.05000000074505806
Edit: Singapore -> Dollar <to> Indonesia -> Rupiah -- found in beta: 0.10000000149011612
Edit: Philippines -> Peso <to> Singapore -> Dollar -- found in beta: 0.15000000596046448
Edit: New Zealand -> Dollar <to> United Arab Emirates -> Dirham -- found in beta: 0.15000000596046448
Edit: Switzerland -> Franc <to> United Kingdom -> Pound -- found in beta: 0.15000000596046448
Edit: United Kingdom -> Pound <to> Switzerland -> Franc -- found in beta: 0.4000000059604645


In [33]:
print("No target match found in faithfulness successes for the following:")
for sample in efficacy_successes.values():
    print(f"Edit: {sample.source} <to> {sample.target}")

No target match found in faithfulness successes for the following:
Edit: Japan -> Yen <to> Hungary -> Forint
Edit: United Arab Emirates -> Dirham <to> Norway -> Krone


In [34]:
train_samples = layer_result.result.samples
train_samples

[RelationSample(subject='Brazil', object='Real'),
 RelationSample(subject='Argentina', object='Peso'),
 RelationSample(subject='Russia', object='Ruble'),
 RelationSample(subject='Poland', object='Zloty'),
 RelationSample(subject='India', object='Rupee'),
 RelationSample(subject='China', object='Yuan'),
 RelationSample(subject='South Korea', object='Won'),
 RelationSample(subject='Canada', object='Dollar'),
 RelationSample(subject='Turkey', object='Lira'),
 RelationSample(subject='Thailand', object='Baht')]

In [35]:
prompt_template = relation_result.trials[TRIAL_NO].prompt_template
prompt_template

' {} :'

In [36]:
from src import functional, operators, editors

In [37]:
estimator = operators.JacobianIclMeanEstimator(
    mt = mt,
    h_layer = LAYER,
)

operator = estimator(
    relation.set(
        samples = train_samples,
        prompt_templates = [prompt_template],
    )
)

In [38]:
############################################
subject = "Hungary"
############################################

In [41]:
# model predicts correctly
functional.predict_next_token(
    mt = mt,
    prompt = operator.prompt_template.format(subject)
)

[[PredictedToken(token=' For', prob=0.970029890537262),
  PredictedToken(token=' Ft', prob=0.0120215630158782),
  PredictedToken(token=' for', prob=0.003609527600929141),
  PredictedToken(token=' F', prob=0.0023671858943998814),
  PredictedToken(token=' Kor', prob=0.0018725981935858727)]]

In [42]:
# LRE fails (low faithfulness)
operator(subject=subject)

LinearRelationOutput(predictions=[PredictedToken(token=' New', prob=0.4707508981227875), PredictedToken(token=' Dollar', prob=0.1228027194738388), PredictedToken(token=' Euro', prob=0.08984438329935074), PredictedToken(token=' ', prob=0.03573762625455856), PredictedToken(token=' Pes', prob=0.034101083874702454)], h=tensor([[-0.3643, -0.0974, -0.7529,  ...,  1.1895, -0.2173,  0.6777]],
       device='cuda:0', dtype=torch.float16), z=tensor([[ 0.5054, -0.1365, -2.2422,  ..., -0.3159,  2.8867,  1.0400]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>))

In [43]:
svd = torch.svd(operator.weight.float())
editor = editors.LowRankPInvEditor(
    lre = operator,
    rank = rank_result.rank,
    n_samples=1, n_new_tokens=1,
    svd = svd
)

In [44]:
efficacy_test_pair = efficacy_successes[subject]
f"Editing: {efficacy_test_pair.source} <to> {efficacy_test_pair.target}"

'Editing: Japan -> Yen <to> Hungary -> Forint'

In [45]:
# editing succeeds (high efficacy)
editor(
    subject = efficacy_test_pair.source.subject,
    target = efficacy_test_pair.target.subject,
)

LinearRelationEditResult(predicted_tokens=[PredictedToken(token=' For', prob=0.8955066204071045), PredictedToken(token=' H', prob=0.0679834708571434), PredictedToken(token=' Kor', prob=0.009948180057108402)], model_logits=tensor([-inf, -inf, -inf,  ..., -inf, -inf, -inf], device='cuda:0'), model_generations=[' Brazil : Real\n Argentina : Peso\n Russia : Ruble\n Poland : Zloty\n India : Rupee\n China : Yuan\n South Korea : Won\n Canada : Dollar\n Turkey : Lira\n Thailand : Baht\n Japan : For'])